Based on the Prototype 2 (AD-PMM) algorithm of Shefi and Teboulle 2014, for convex QPs of the form:
$$
\begin{align}
\text{mimimise } &\frac{1}{2} x^T P x + c^T x \\
\text{subject to } & Ax + s = b \\
& s \in \mathcal{K}
\end{align}
$$
where $P \in \mathbb{S}^{n}_+$, $c \in \mathbb{R}^n$, $A \in \mathbb{R}^{m \times n}$, and $b \in \mathbb{R}^m$.

For now I only have code for $\mathcal{K} = \mathbb{R}^n_+$. Julia code for projections onto many cones of interest is available in the COSMO stuff.


For a generic square symmetric matrix $P$, we use the notation/partition
$$
P = D(P) + R(P),
where $D(P)$ is diagonal with the same entries as those in the main diagonal of $P$. $R(P)$ is the remainder, entries not in the diagonal of $P$.

In [8]:
# Get the required libraries
using LinearAlgebra
using Printf

In [9]:
# Define the QP problem data
struct QPProblem
    P::Matrix{Float64}  # PSD matrix in R^{n x n}
    c::Vector{Float64}  # Vector in R^n
    A::AbstractMatrix{Float64}  # Matrix in R^{m x n}
    b::Vector{Float64}  # Vector in R^m
    D::Diagonal{Float64, Vector{Float64}}
    R::Matrix{Float64}
    M1::Matrix{Float64}
    τ::Float64
    ρ::Float64
    
    
    # Constructor with a check for P being PSD
    function QPProblem(P::Matrix{Float64}, c::Vector{Float64}, A::Matrix{Float64}, b::Vector{Float64}, τ::Float64, ρ::Float64)
        # Check if P is symmetric and positive semidefinite
        if !issymmetric(P)
            error("Matrix P must be symmetric.")
        end
        
        # Check positive semidefiniteness of P by verifying all eigenvalues are non-negative
        if minimum(eigvals(P)) < 0
            error("Matrix P must be positive semidefinite (PSD).")
        end

        # Precompute data for iterate updates
        D = Diagonal(diag(P))
        R = P - D
        
        # Check for M1 being PSD
        n = size(A)[2]
        M1 = - (R + ρ * A' * A) + I(n) / τ
        if minimum(eigvals(M1)) < 0
            error("Matrix M1 must be PSD. Current smallest eigval: $minimum(eigvals(M1))")
        end
        
        # Create the instance of the struct
        new(P, c, A, b, D, R, M1, τ, ρ) # what to store in struct, to be accessed later
    end
end

In [10]:
# Initialise QP data
n = 3  # number of variables
m = 3  # number of (inequality) constraints

P = [5.0 0 1; 0 6 6; 1 6 8]
c = [1.0, 2, 3]
b = -1 * ones(Float64, m)
A = Matrix{Float64}(I, n, n)

# Step size parameters chosen by the user
τ = 0.1 # (primal)
ρ = 1.0; # (dual)

# Solve with our method

In [12]:
#TODO: implement residual calculation
# termination criteria; look at COSMO paper.

# Can later try against SCS or COSMO

# mul! function for multiplication? or better to just count iterations for now (code is more readable)
# benchmarks from Clarabel paper --- need to extract problem data, maybe tricky

# Create a problem instance
problem = QPProblem(P, c, A, b, τ, ρ)

# Initialize sequences x, z, and y
x = ones(n)
z = ones(m)
y = ones(m)

# Define the iteration function
function iterate!(problem::QPProblem, x::Vector{Float64}, z::Vector{Float64}, y::Vector{Float64})
    # Unpack problem data for easier reference
    P, c, A, b, τ, ρ, D, R, M1 = problem.P, problem.c, problem.A, problem.b, problem.τ, problem.ρ, problem.D, problem.R, problem.M1
    
    ### Step 1: Update x ###
    # Solve diagonal PD system
    # x_new = (-A' * (y - ρ * z) - c + M1 * x) ./ diag(D + (1/τ) * I(n))
    x_new = 

    ### Step 2: Update z ###
    # Project basic update onto {z | z ⪯ b}
    z_new = min.(A * x_new + y / ρ, b)

    ### Step 3: Update y ###
    y_new = y + ρ * (A * x_new - z_new)

    # Update the sequences
    x .= x_new
    z .= z_new
    y .= y_new
end

# Main iteration loop
max_iters = 200  # Maximum number of iterations (adjust as needed)
for k in 0:max_iters
    # Print each iteration result with formatted output on a single line
    if k % 10 == 0
        @printf("Iteration %4.1d: x = [%s]  z = [%s]  y = [%s]\n",
            k,
            join(map(xi -> @sprintf("%12.5e", xi), x), ", "),
            join(map(zi -> @sprintf("%12.5e", zi), z), ", "),
            join(map(yi -> @sprintf("%12.5e", yi), y), ", "))
    end
    
    iterate!(problem, x, z, y)
end

# Print final solution on a single line
println("\nPrototype FOM results:")
@printf("x = [%s]  z = [%s]  y = [%s]\n",
    join(map(xi -> @sprintf("%12.5e", xi), x), ", "),
    join(map(zi -> @sprintf("%12.5e", zi), z), ", "),
    join(map(yi -> @sprintf("%12.5e", yi), y), ", "))

Iteration    0: x = [ 1.00000e+00,  1.00000e+00,  1.00000e+00]  z = [ 1.00000e+00,  1.00000e+00,  1.00000e+00]  y = [ 1.00000e+00,  1.00000e+00,  1.00000e+00]
Iteration   10: x = [-1.04787e+00, -7.92824e-01, -5.00826e-01]  z = [-1.00000e+00, -1.00000e+00, -1.00000e+00]  y = [ 4.87419e+00,  6.09963e+00,  6.62393e+00]
Iteration   20: x = [-9.83585e-01, -8.18042e-01, -8.34562e-01]  z = [-1.00000e+00, -1.00000e+00, -1.00000e+00]  y = [ 4.65865e+00,  7.62724e+00,  9.93654e+00]
Iteration   30: x = [-9.85195e-01, -9.04242e-01, -9.33327e-01]  z = [-1.00000e+00, -1.00000e+00, -1.00000e+00]  y = [ 4.89329e+00,  9.21742e+00,  1.07247e+01]
Iteration   40: x = [-9.99916e-01, -9.91393e-01, -9.35039e-01]  z = [-1.00000e+00, -1.00000e+00, -1.00000e+00]  y = [ 4.92403e+00,  9.50833e+00,  1.14950e+01]
Iteration   50: x = [-9.94271e-01, -9.67277e-01, -9.94277e-01]  z = [-1.00000e+00, -1.00000e+00, -1.00000e+00]  y = [ 4.96455e+00,  9.76247e+00,  1.17675e+01]
Iteration   60: x = [-1.00003e+00, -9.97600e-0

# Solve with OSQP

In [13]:
using OSQP
using SparseArrays

# Convert problem data to sparse type
P_osqp = sparse(P)
A_osqp = sparse(A)
# Create OSQP object
prob = OSQP.Model()
# Set up workspace and change alpha parameter
OSQP.setup!(prob; P=P_osqp, q=c, A=A_osqp, u=b, alpha=1, verbose=false)
# Solve problem
results = OSQP.solve!(prob);

# Print OSQP results
println("OSQP results:")
println("x = [", join(map(v -> @sprintf("%12.5e", v), results.x), ", "), "]")
println("y = [", join(map(v -> @sprintf("%12.5e", v), results.y), ", "), "]")

OSQP results:
x = [-1.00000e+00, -1.00000e+00, -1.00000e+00]
y = [ 5.00000e+00,  1.00000e+01,  1.20000e+01]
